# LangChain: Prompts, Chains, and Message History

This notebook covers:
- **Prompt Templates** - SystemMessage, HumanMessage, ChatPrompt with variables
- **Output Parsers** - StrOutputParser for clean text output
- **Chains** - Combining templates, models, and parsers
- **Parallel Chains** - RunnableParallel for concurrent execution
- **Chain Routing** - Conditional branching based on input
- **Lambda Chains** - Inline transformations
- **Passthrough** - Preserving original data through chains
- **Message History** - Maintaining conversation context with SQLChatMessageHistory
- **Custom Functions** - Building reusable chat functions

Works with both **Anthropic Claude** and **Ollama** models!

## Setup and Installation

```bash
pip install langchain-anthropic langchain-ollama langchain-core langchain-community python-dotenv
```

In [ ]:
# Import required libraries
from dotenv import load_dotenv
import os
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableParallel,
    RunnableLambda,
    RunnablePassthrough,
    chain
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama

# Load environment variables
load_dotenv('../.env')

print("✅ All imports successful!")

## Model Configuration

Toggle between Anthropic Claude and Ollama models.

In [ ]:
# Model selection
USE_OLLAMA = False  # Set to True for Ollama, False for Anthropic

if USE_OLLAMA:
    model = ChatOllama(
        model="llama3.2",
        base_url=os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"),
        temperature=0.7,
    )
    print("🦙 Using Ollama (llama3.2)")
else:
    model = ChatAnthropic(
        model="claude-3-5-sonnet-20241022",
        temperature=0.7,
        max_tokens=4000,
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )
    print("🤖 Using Anthropic Claude (claude-3-5-sonnet-20241022)")

# Part 1: Prompt Templates

## What are Prompt Templates?

Prompt templates allow you to create reusable prompts with variables. Instead of hardcoding prompts, you can create templates with placeholders that get filled in at runtime.

### Benefits:
- **Reusability** - Write once, use many times
- **Consistency** - Same prompt structure across use cases
- **Maintainability** - Change prompts in one place
- **Type Safety** - Define expected variables

## Example 1: SystemMessage and HumanMessage Templates

In [ ]:
# Create a system message template with a role
system = SystemMessagePromptTemplate.from_template(
    "You are a {role}. Your job is to help and guide junior developers."
)

# Create a human message template with topic variable
question = HumanMessagePromptTemplate.from_template(
    "Explain {topic} with its real-world application and trade-offs if any."
)

# Display the templates
print("System Template:", system)
print("\nQuestion Template:", question)

# Format with actual values
formatted_question = question.format(topic='RDD')
print("\nFormatted Question:", formatted_question)

## Example 2: ChatPromptTemplate

Combine multiple message templates into a single chat prompt.

In [ ]:
# Create a chat prompt from the messages
messages = [system, question]
template = ChatPromptTemplate.from_messages(messages)

print("Chat Template:", template)
print("\nInput Variables:", template.input_variables)

## Example 3: Using the Template with a Model

In [ ]:
# Invoke the template with variables to create a full prompt
question_prompt = template.invoke({
    'role': 'AWS Cloud Architect',
    'topic': 'Amazon RedShift'
})

# Send to the model
response = model.invoke(question_prompt)
print(response.content)

# Part 2: Chains and Output Parsers

## What are Chains?

Chains connect multiple components together:
- **Prompt Template** → **Model** → **Output Parser**

Using the `|` operator (pipe), we can chain components elegantly.

## StrOutputParser

Extracts just the text content from model responses, removing metadata.

## Example 4: Basic Chain with StrOutputParser

In [ ]:
# Create a chain: template | model | parser
chains = template | model

# Invoke the chain
result = chains.invoke({'role': 'Data Engineer', 'topic': 'Data Lakes'})
print(result.content)

## Example 5: Chain with StrOutputParser

In [ ]:
# Add output parser to get clean string output
from langchain_core.output_parsers import StrOutputParser

chain = template | model | StrOutputParser()

# Now response is just a string
response = chain.invoke({'role': 'DevOps Engineer', 'topic': 'Infrastructure as Code'})
print(response)

# Part 3: Composed Chains and Summarization

Chain outputs can feed into other chains, enabling complex workflows.

## Example 6: Summarization Chain

In [ ]:
# Create a summarization prompt
summarization_prompt = ChatPromptTemplate.from_template(
    """
    Your job is to analyze the text provided to you and provide
    a concise summary not more than 100 words.
    Text = {response}
    """
)

# Create summarization chain
summarization_chain = summarization_prompt | model | StrOutputParser()

# Use the output from previous chain
output = summarization_chain.invoke(response)
print(output)

## Example 7: Composed Chain Pattern

In [ ]:
# Combine both chains into one
# First get detailed response, then summarize it
composed_chain = {"response": chain} | summarization_chain

final_output = composed_chain.invoke({
    'role': 'Software Engineer',
    'topic': 'Generative AI using Bedrock'
})

print(final_output)

# Part 4: Guidance and Leadership Chains

Create specialized chains for different perspectives.

## Example 8: Multiple Perspective Chains

In [ ]:
# Question template for best practices
q1 = HumanMessagePromptTemplate.from_template(
    'What are the best practices for implementing the {topic} in {technology}?'
)

# Build guidance chain
msg1 = [system, q1]
chat_template1 = ChatPromptTemplate.from_messages(msg1)
guidance_chain = chat_template1 | model | StrOutputParser()

# Test guidance chain
output1 = guidance_chain.invoke({
    'role': 'Principal Architect',
    'topic': 'Designing APIs for Banking System',
    'team': 'Backend Development team',
    'technology': 'Python'
})

print("\n=== GUIDANCE RESPONSE ===")
print(output1)

## Example 9: Leadership Perspective Chain

In [ ]:
# Question template for leadership best practices
q2 = HumanMessagePromptTemplate.from_template(
    'As a leader, what are some best practices you would recommend for {team} when working on {topic} using {technology}?'
)

# Build leadership chain
msg2 = [system, q2]
chat_template2 = ChatPromptTemplate.from_messages(msg2)
leadership_chain = chat_template2 | model | StrOutputParser()

# Test leadership chain
output2 = leadership_chain.invoke({
    'role': 'Principal Architect',
    'topic': 'Designing APIs for Banking System',
    'team': 'Backend Development team',
    'technology': 'Python'
})

print("\n=== LEADERSHIP RESPONSE ===")
print(output2)

# Part 5: RunnableParallel - Concurrent Execution

Run multiple chains in parallel and get all results at once.

**Benefits:**
- Faster execution (parallel vs sequential)
- Get multiple perspectives simultaneously
- Structured output format

## Example 10: Parallel Chain Execution

In [ ]:
# Run both chains in parallel
chain_parallel = RunnableParallel(
    guidance=guidance_chain,
    leadership=leadership_chain
)

# Execute both chains at once
parallel_result = chain_parallel.invoke({
    'role': 'Principal Architect',
    'topic': 'Designing APIs for Banking System',
    'team': 'Backend Development team',
    'technology': 'Python'
})

print("\n=== PARALLEL EXECUTION RESULTS ===")
print("\n📋 Response from Guidance Chain:")
print(parallel_result['guidance'])
print("\n👔 Response from Leadership Chain:")
print(parallel_result['leadership'])

## Example 11: Using @chain Decorator

In [ ]:
# Define a custom chain using decorator
@chain
def custom_chain(params):
    return {
        'guidance': guidance_chain.invoke(params),
        'leadership': leadership_chain.invoke(params)
    }

# Test custom chain
params = {
    'role': 'Principal Architect',
    'topic': 'Designing APIs for Banking System',
    'team': 'Backend Development team',
    'technology': 'Python'
}

output = custom_chain.invoke(params)
print('Response from Guidance Chain:')
print(output['guidance'])
print('\n Response from Leadership Chain:')
print(output['leadership'])

# Part 6: Chain Router - Conditional Branching

Route to different chains based on input conditions.

**Use Cases:**
- Sentiment analysis → different response templates
- Classification → different processing pipelines
- Priority routing → different handlers

## Example 12: Sentiment-Based Routing

In [ ]:
# Create sentiment classifier prompt
prompt = """
Given the user review below, classify it as either being about 'Positive' or 'Negative'.
Do not respond with more than one word.

Review: {review}
Classification:
"""

template = ChatPromptTemplate.from_template(prompt)
chain = template | model | StrOutputParser()

# Test sentiment classification
review = "The product quality is excellent and the customer service was very helpful."
sentiment = chain.invoke({'review': review})
print(f"Sentiment: {sentiment}")

## Example 13: Creating Response Chains for Each Sentiment

In [ ]:
# Positive review response template
positive_prompt = """
You are expert in writing reply for positive reviews.
You need to encourage the customer to share their experience on social media
and ask them to recommend the product to their friends and family.
Review: {review}
Reply:
"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | model | StrOutputParser()

# Negative review response template
negative_prompt = """
You are expert in writing reply for negative reviews.
You need to apologize for the inconvenience caused and offer a solution to resolve the issue.
You need to encourage the customer to share their concern on the following
email address: 'support@example.com'.
Review: {review}
Reply:
"""

negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | model | StrOutputParser()

print("✅ Response chains created")

## Example 14: Route Function

In [ ]:
# Define routing function
def route(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

# Test routing
test_info = {'sentiment': 'Positive', 'review': review}
selected_chain = route(test_info)
print(f"Selected chain: {selected_chain}")

## Example 15: Complete Routing Chain with RunnableLambda

In [ ]:
from langchain_core.runnables import RunnableLambda

# Build complete routing chain
full_chain = {
    'sentiment': chain,
    'review': lambda x: x['review']
} | RunnableLambda(route) | StrOutputParser()

# Test with positive review
review1 = "The product quality is excellent and the customer service was very helpful."
response1 = full_chain.invoke({'review': review1})
print("\n=== POSITIVE REVIEW RESPONSE ===")
print(response1)

# Test with negative review
review2 = "The product stopped working after a week and the customer service was unresponsive."
response2 = full_chain.invoke({'review': review2})
print("\n=== NEGATIVE REVIEW RESPONSE ===")
print(response2)

# Part 7: RunnablePassthrough

Pass data through a chain without modification, useful for preserving context.

**Use Cases:**
- Keep original input alongside transformations
- Add metadata to chain outputs
- Debugging and logging

## Example 16: Helper Functions for Analysis

In [ ]:
# Helper functions
def count_words(text):
    return len(text.split())

def count_chars(text):
    return len(text)

# Test functions
test_text = "Hello world, this is a test"
print(f"Words: {count_words(test_text)}")
print(f"Characters: {count_chars(test_text)}")

## Example 17: Chain with Passthrough and Analysis

In [ ]:
# Create a chain that includes analysis
prompt = ChatPromptTemplate.from_template(
    "Explain the concept of {topic} in {technology}. Keep the explanation concise and max 3 lines."
)

chain = prompt | model | StrOutputParser() | {
    'word_count': lambda x: count_words(x),
    'char_count': lambda x: count_chars(x),
    'output': RunnablePassthrough()
}

# Execute chain
output = chain.invoke({'topic': 'Async', 'technology': 'Python'})
print(output)

# Part 8: Message History and Conversation Memory

Maintain conversation context across multiple interactions.

**Components:**
- `SQLChatMessageHistory` - Store messages in SQLite database
- `RunnableWithMessageHistory` - Automatically manage conversation history
- `MessagesPlaceholder` - Template variable for message history

**Benefits:**
- Context-aware responses
- Multi-turn conversations
- Persistent memory

## Example 18: Basic Conversation (No Memory)

In [ ]:
# Create a simple chat chain
template = ChatPromptTemplate.from_template("{prompt}")
chain = template | model | StrOutputParser()

# First query
query = "I am currently working as a Generative AI Engineer at Vanguard."
response1 = chain.invoke({'prompt': query})
print("Response 1:", response1)

# Second query (no memory - model won't remember previous message)
response2 = chain.invoke({'prompt': "What is my current job title?"})
print("\nResponse 2:", response2)

## Example 19: Setting Up Message History

In [ ]:
# Function to get or create message history for a session
def get_session_history(session_id: str) -> SQLChatMessageHistory:
    """Fetch message history for a given session ID from SQL database."""
    return SQLChatMessageHistory(session_id, "sqlite:///chat_history.db")

# Create template with history placeholder
template = ChatPromptTemplate.from_template("{prompt}")
chain = template | model | StrOutputParser()

# Wrap chain with message history
runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history
)

print("✅ Message history configured")

## Example 20: Conversation with Memory

In [ ]:
# User session ID
user_id = 'aniket_0123'

# Get history for this user
history = get_session_history(user_id)

# First message
runnable_with_history.invoke(
    [HumanMessage(content="I am currently working as a Generative AI Engineer at Vanguard.")],
    config={"configurable": {"session_id": user_id}}
)

# View message history
print("\n=== Message History ===")
messages = history.get_messages()
for msg in messages:
    print(f"{type(msg).__name__}: {msg.content[:100]}...")

## Example 21: Follow-up Questions with Context

In [ ]:
# Ask follow-up question (model now has context)
response = runnable_with_history.invoke(
    [HumanMessage(content="What is my current job title?")],
    config={"configurable": {"session_id": user_id}}
)

print("\n=== Response with Context ===")
print(response)

# Check updated history
print("\n=== Updated Message History ===")
messages = history.get_messages()
for i, msg in enumerate(messages, 1):
    print(f"{i}. {type(msg).__name__}: {msg.content[:80]}...")

## Example 22: Advanced - History with System Prompt

In [ ]:
# Create template with system message and history
system = SystemMessagePromptTemplate.from_template(
    template="You are a helpful assistant."
)
human = HumanMessagePromptTemplate.from_template(
    template="{input}"
)

messages = [
    system,
    MessagesPlaceholder(variable_name="history"),
    human
]

prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | model | StrOutputParser()

runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='input',
    history_messages_key='history'
)

print("✅ Advanced chain with history configured")

## Example 23: Custom Chat Function

In [ ]:
def chat_with_llm(user_input: str, session_id: str) -> str:
    """Chat with LLM while maintaining message history."""
    return runnable_with_history.invoke(
        {"input": user_input},
        config={"configurable": {"session_id": session_id}}
    )

# Test the chat function
user_id = "user-1254"

response1 = chat_with_llm("My name is Gpt Smith, and I make AI models.", user_id)
print("Response 1:", response1)

response2 = chat_with_llm("What is my name?", user_id)
print("\nResponse 2:", response2)

response3 = chat_with_llm("What do I do for work?", user_id)
print("\nResponse 3:", response3)

# Summary: Key Concepts

## 1. Prompt Templates
- **SystemMessagePromptTemplate** - Define AI role/behavior
- **HumanMessagePromptTemplate** - User message with variables
- **ChatPromptTemplate** - Combine multiple messages

## 2. Chains
- **Basic Chain** - `template | model | parser`
- **Composed Chains** - Output of one chain feeds into another
- **Parallel Chains** - `RunnableParallel` for concurrent execution

## 3. Routing
- **Conditional Logic** - Route to different chains based on input
- **RunnableLambda** - Custom transformation functions

## 4. Data Flow
- **RunnablePassthrough** - Preserve original data
- **Lambda Functions** - Inline data transformations

## 5. Message History
- **SQLChatMessageHistory** - Persistent conversation storage
- **RunnableWithMessageHistory** - Automatic history management
- **MessagesPlaceholder** - Template variable for history

## Best Practices

1. **Use Templates** - Makes prompts reusable and maintainable
2. **Chain Composition** - Build complex workflows from simple components
3. **Output Parsing** - Use `StrOutputParser()` for clean text output
4. **Parallel Execution** - Use `RunnableParallel` for multiple perspectives
5. **Routing** - Branch logic based on classification/sentiment
6. **Message History** - Essential for multi-turn conversations
7. **Session Management** - Use unique session IDs for different users

## Production Considerations

- **Error Handling** - Wrap chains in try-except blocks
- **Rate Limiting** - Respect API limits
- **Cost Management** - Monitor token usage
- **Database** - Use PostgreSQL for production (not SQLite)
- **Logging** - Track conversations for debugging
- **Security** - Sanitize user inputs
- **Privacy** - Handle sensitive data appropriately